In [1]:
import asyncio
import os
import sys
from dotenv import load_dotenv
import tqdm
from concurrent.futures import ProcessPoolExecutor
import nest_asyncio
import logging
import math
from typing import List, Tuple
import pandas as pd



logging.getLogger("asyncio").setLevel(logging.WARNING)
nest_asyncio.apply()

In [ ]:
import asyncio
from concurrent.futures import ProcessPoolExecutor
from itertools import islice
import time
import nest_asyncio
nest_asyncio.apply()

async def asy(x):
    print('enter: ', x)
    await asyncio.sleep(1)
    print('finishing ', x)
    return x

def run(corofn, *args):
    loop = asyncio.new_event_loop()
    try:
        coro = corofn(*args)
        asyncio.set_event_loop(loop)
        ret = loop.run_until_complete(coro)
        #print(ret)
        return ret
    finally:
        loop.close()

def limit_futures(futures, limit):
    futures_sl = [
        c for c in islice(futures, 0, limit)
    ]
    print(len(futures_sl))
    async def first_to_finish(futures):
        while True:
            await asyncio.sleep(0)
            for f in futures_sl:
                if f.done():
                    futures_sl.remove(f)
                    try:
                        newf = next(futures)
                        futures_sl.append(newf)
                    except StopIteration as e:
                        pass
                    return f.result()
    while len(futures_sl) > 0:
        yield first_to_finish(futures)

async def main():
    loop = asyncio.get_event_loop()
    executor = ProcessPoolExecutor()
    
    futures = (loop.run_in_executor(executor, run, asy, x) for x in range(110000,119990))
    '''
    CASE balls to the wall!
    
    '''
    await asyncio.gather(*futures)
    for ret in limit_futures(futures, 4): # limitation - 4 per all processes
        await ret

if __name__ == '__main__':
    start = time.time()
    '''
    # CASE single
    ret = [asy(x) for x in range(510000,510040)]
    exit()
    '''
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
    print("Elapsed time: {:.3f} sec".format(time.time() - start))

In [ ]:
async def fetch_price(
    ticker: str, client: httpx.AsyncClient
) -> Tuple[str, Decimal]:
    print(f"Making request for {ticker} price")
    response = await client.get(YAHOO_FINANCE_URL.format(ticker))
    print(f"Received results for {ticker}")
    price = response.json()["chart"]["result"][0]["meta"]["regularMarketPrice"]
    return ticker, Decimal(price).quantize(Decimal("0.01"))


async def fetch_all_prices(tickers: List[str]) -> List[Tuple[str, Decimal]]:
    async with httpx.AsyncClient() as client:
        return await asyncio.gather(
            *map(fetch_price, tickers, itertools.repeat(client),)
        )

loop = asyncio.get_event_loop()
result = loop.run_until_complete(fetch_all_prices())

In [ ]:
logging.getLogger("asyncio").setLevel(logging.WARNING)

nest_asyncio.apply()

class async_runner(object):
    def __init__(self):
        self.futures = [] # container to store current futures
        self.futures_total = []
        self.loop = asyncio.get_event_loop() # main event_loop
        self.executor = ProcessPoolExecutor()
        self.limit = 1

    @staticmethod 
    def run(self, func, *args):
        temp_loop = asyncio.new_event_loop()
        try:
            coro = func(*args)
            asyncio.set_event_loop(temp_loop)
            ret = temp_loop.run_until_complete(coro)
            return ret
        finally:
            temp_loop.close()
    
    def limit_futures(self, futures, limit):
        self.futures_total = iter(futures)
        self.futures = [future for future in islice(self.futures_total, 0, limit)]
        #return [future for future in islice(self.futures_total, 0, limit)]
        async def first_to_finish():
            for f in tqdm.asyncio.tqdm.as_completed(self.futures):
                await f
                print('qqwe')
                if f.done(): # here raised TypeError: can't pickle coroutine objects
                    print(f.done())
                    print('asdf')
                    self.futures.remove(f)
                    try:
                        print(f.done())
                    except StopIteration as e:
                        print('lalslasd')
                        pass
                    
                    return f.result()
        while len(self.futures) > 0:
            yield first_to_finish()
    
    async def run_limited(self, func, args, limit):
        self.limit = int(limit)
        self.futures_total = (self.loop.run_in_executor(self.executor, func, x) for x in range(110000,119990))
        self.futures = [future for future in islice(self.futures_total, 0, self.limit)]
        
        for f in tqdm.asyncio.tqdm.as_completed(self.futures):
            f
            print('s')
            
    def set_execution(self, func, args, limit):
        future = asyncio.run_coroutine_threadsafe(self.run_limited(func, args, limit), loop=self.loop)
        return future.result()

async def asy(x):
    print('enter: ', x)
    await asyncio.sleep(randint(1,3))
    print('finishing ', x)
    return x

runner = async_runner()
ret = runner.set_execution(asy, 'h', 3)
print(ret)